### Demonstration of Stern-Gerlach simulator of a particle on a ring/2D rigid rotor with alternating measurements of position and energy

Brief outline of the simulation:

- System is initialized as a Gaussian wavepacket localized about an angle of orientation $\phi_0$ with average angular momentum $\hbar m_0$

$$\Psi(\phi, t=0) = \frac{1}{\sigma \sqrt{2 \pi}} \: {\rm exp} \left( -\frac{1}{2} \left( \frac{\phi- \phi_0}{\sigma} \right)^2 \right) \: {\rm exp} \left(i m_0 \phi \right) $$

- The wavepacket is expanded in the basis of 2D rigid rotor energy eigenfunctions permitting time-evolution of the initial state

$$ \Psi(\phi, t) = \sum_{m=-m_{max}}^{m_{max}} c_m \frac{1}{\sqrt{2\pi}}  {\rm exp}\left( i m \phi \right) {\rm exp}\left( -\frac{i}{\hbar} E_m t  \right) $$

where the coefficients are determined by 

$$ c_m = \int_0^{2\pi} \frac{1}{\sqrt{2\pi}}  {\rm exp}\left( - i m \phi \right) \Psi(\phi, t_0=0) \: dx. $$

and the energy eigenvalues are given by 

$$ E_m = \frac{\hbar^2 m^2}{2 I} = \frac{\hbar^2 m^2}{2 \mu r^2} $$

where $\mu$ indicates the reduced mass and $r$ is the radius of the axis of rotation.  We pick $\mu$ and $r$ to correspond approximately to the electron mass and the radius of a benzene ring, so the simulation below will correspond most physically to the motion of an aromatic electron.

- Measurement of position at $t = 50$ is simulated by choosing a random $\phi$ value weighted by the probability density; we will call the measured position $\phi_{collapse}$.

$$ P(\phi, 50) = \left|\Psi(\phi, 50)\right|^2 =  \left|\sum_{m=-m_{max}}^{m_{max}} c_m \frac{1}{\sqrt{2\pi}}  {\rm exp}\left( i m \phi \right) {\rm exp}\left( -\frac{i}{\hbar} E_m \cdot 50  \right)\right|^2 $$

- The resulting wavefunction should be a position eigenstate located at $\phi = \phi_{collapse}$, which is approximated by a Gaussian wavepacket with a small spread in $\phi$ and zero average angular momentum.

- New expansion coefficients $d_m$ for the collapsed state are determined, and again the time evolution is determined from the known time-dependence of each energy eigenfunction

- Measurement of energy at $t=150$ is simulated by choosing a random quantum number $m$ weighted by the probabilities encoded by $P(m) = \left| d_m \right|^2$, resulting in an energy eigenvalue $E_m$ and collapse to an energy eigenstate $\psi_m(\phi, t) = \frac{1}{\sqrt{2\pi}} {\rm exp}\left(i m \phi \right){\rm exp}\left(-\frac{i}{\hbar} E_m t \right)$.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc
from IPython.display import HTML

# set up figure, the axis, and the plot elements 
fig, ax = plt.subplots()
plt.close()

### Necessary parameters for the particle on a ring 
### Radius in atomic units... electron mass is 1 atomic unit of mass
mu = 1 
# radius for benzene reference Heyrovska et al 2008
R = 5.29 
# moment of inertia
I = mu * R**2
hbar = 1
# domain of the polar angle phi
L = 2*np.pi


### parameters that define the 
### resolution of the spacial grid
### and the number of energy eigenfunctios
### to use in the expansion of any given wavefunction
### throughout the simulation
phi_grid_points = 200

# maximum quantum number determines the
# highest energy eigenfunction that will
# be used in the expansion
max_m = 50
# number of basis functions will be max_n + 1
# since the ground state energy eigenfunction has n = 0
# for the harmonic oscillator
num_basis_functions = 2 * max_m + 1

### create a numpy array for the polar angle
## Appropriate range - 0 -> 2pi
x = np.linspace(0, L, phi_grid_points)


### make an array of quantum numbers
m_array = np.linspace(-max_m, max_m, num_basis_functions)

### create an empty array of coefficients with 500 enteries 
cn = np.zeros(num_basis_functions, dtype=complex)

#parameters for plot -> change to appropriate range
ax.set_xlim((0, L))
ax.set_ylim((-5, 5))

line, = ax.plot([], [], lw=2)


def init():
    line.set_data([], [])
    return (line,)

def gaussian_packet(phi_0, sig, k0, phi):
    ci = 0 + 1j
    T1 = 1 / (sig * np.sqrt(2 * np.pi))
    T2 = np.exp((-0.5) * ((phi - phi_0)/sig)**2)
    T3 = np.exp(ci * k0 * phi)
    return T1 * T2 * T3

### Particle-on-a-ring function!
def energy_eigenfunction(m, phi):
    ### normalize!
    return np.sqrt(1/(2*np.pi)) * np.exp(1j * m * phi)

def energy_eigenvalue(m, I):
    return hbar**2 * m**2 / ( 2 * I )

### return time-dependent part of energy eigenfunction in atomic units 
def time_component(m, I, t):
    ci = 0+1j
    Em = energy_eigenvalue(m, I)
    return np.exp(-ci*Em*t)

### position eigenfunction generator!
def position_eigenfunction(phi_0, sig, phi):
    ### need sqrt of -1!
    ci = 0+1j
    ### T1 will be the prefactor that is 1/(sigma * sqrt(2 * pi))
    T1 = 1/(sig * np.sqrt(2 * np.pi))
    ### T2 will be the Gaussian function, exp(-0.5 * ((x-x0)/sigma)^2)
    T2 = np.exp(-0.5 * ((phi-phi_0)/sig)**2)
    ### T3 will be the complex exponential (aka the plane wave!)
    ### return the product of T1 * T2 * T3
    return T1 * T2 

### x0 and sig to appropriate values for the particle-on-a-ring system 
x0 = np.pi 
k0 = 0.1
sig = 0.1

# initial state
Psi = gaussian_packet(x0, sig, k0, x)

# array to store basis set expansion of initial state
Psi_expanded = np.zeros_like(Psi)
for i in range(0, num_basis_functions):
    
    ### 1. the function you want to integrate against comes from phi_m^* * Psi
    phi_m = energy_eigenfunction(m_array[i], x)
    integrand = np.conj(phi_m) * Psi
    
    cn[i] = np.trapz(integrand, x)
    Psi_expanded = Psi_expanded + cn[i] * phi_m

    


N_time = 200
n0 = np.zeros(1)
# animate function. this is called sequentially 
def animate(i):
    y = np.zeros(len(x),dtype=complex)
    if i<50:
        for j in range(0, num_basis_functions):
            ft = time_component(m_array[j], I, i/10)
            fx = energy_eigenfunction(m_array[j], x)
            y= y + cn[j] * fx * ft 

    ### make position measurement!
    elif i==50:
        for j in range(0, num_basis_functions):
            ft = time_component(m_array[j], I, i/10)
            fx = energy_eigenfunction(m_array[j], x)
            y = y + cn[j] * fx * ft
            
        ### get probability density
        P = np.real(np.conj(y) * y)
        ### make sure P is noramlized
        norm = np.sum(P)
        P_norm = P / norm
        ### measure position and get position eigenvalue/eigenfunction
        p0 = np.random.choice(x, 1, p=P_norm)
        print(" Position measured to be at ",p0)
        pf = position_eigenfunction(p0[0], 0.05, x)
            
        ### now get NEW expansion coefficients given that the new state is pf!
        for j in range(0,100):
            ### get energy eigenstate n
            psi = energy_eigenfunction(m_array[j], x)
            ft = time_component(m_array[j], I, (i-50)/10)
            ### multiply psi_n by gaussian wavepacket
            integrand = np.conj(psi)*pf
            ### get coefficient c_n from the integral of psi_n * Psi_gp
            cn[j] = np.trapz(integrand, x)
            y = y + cn[j] * psi * ft
    elif i<150:
        for j in range(0,100):
            ft = time_component(m_array[j], I, (i-50)/10)
            fx = energy_eigenfunction(m_array[j], x)
            y = y + cn[j] * fx * ft
            
    ### measure energy!
    elif i==150:
        pn = np.real(np.conj(cn) * cn)
        norm = np.sum(pn)
        pn_norm = pn/norm
        ### get random quantum number 
        nval = np.random.choice(m_array, 1, p=pn_norm)
        n0[0] = nval[0]
        En0 = energy_eigenvalue(n0[0], I)
        print(" Randomly measured state", n0[0], "which has angular momentum ",En0)
        ft = time_component(n0[0], I, (i-150)/10)
        fx = energy_eigenfunction(n0[0], x)
        y = fx * ft
    else:
        ft = time_component(n0[0], I, (i-150)/10)
        fx = energy_eigenfunction(n0[0], x)
        y = fx * ft
        
    line.set_data(x, np.real(y))
    return (line,)

rc('animation', html='jshtml')

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=N_time, interval=100, blit=True)

# Note: below is the part which makes it work on colab 

anim